# A.3 广播

广播（broadcasting）指的是不同形状的数组之间的算术运算的执行方式。它是一种非常强大的功能，但也容易令人误解，即使是经验丰富的老手也是如此。将标量值跟数组合并时就会发生最简单的广播：

In [1]:
import numpy as np

In [4]:
arr = np.arange(5)

arr

array([0, 1, 2, 3, 4])

In [5]:
arr * 4

array([ 0,  4,  8, 12, 16])

这里我们说：在这个乘法运算中，标量值4被广播到了其他所有的元素上。

看一个例子，我们可以通过减去列平均值的方式对数组的每一列进行距平化处理。这个问题解决起来非常简单：

In [16]:
arr = np.random.randn(4, 3)

arr

array([[ 1.46367305e+00,  5.49854060e-01,  1.41297481e+00],
       [ 1.15477989e+00, -8.49189952e-01, -1.95989985e+00],
       [-1.58178574e+00, -7.07576930e-01, -1.89393709e+00],
       [ 9.76919327e-01,  4.48428159e-01,  9.93296455e-04]])

In [17]:
arr.mean()

-0.08206391390045777

In [22]:
arr.mean(0)

demeaned = arr - arr.mean(0)

demeaned

array([[ 0.96027642,  0.68947523,  2.02294202],
       [ 0.65138326, -0.70956879, -1.34993264],
       [-2.08518237, -0.56795576, -1.28396988],
       [ 0.4735227 ,  0.58804932,  0.6109605 ]])

In [23]:
demeaned.mean(0)

array([ 0.00000000e+00,  2.77555756e-17, -5.55111512e-17])

图A-4形象地展示了该过程。用广播的方式对行进行距平化处理会稍微麻烦一些。幸运的是，只要遵循一定的规则，低维度的值是可以被广播到数组的任意维度的（比如对二维数组各列减去行平均值）。

于是就得到了：

虽然我是一名经验丰富的NumPy老手，但经常还是得停下来画张图并想想广播的原则。再来看一下最后那个例子，假设你希望对各行减去那个平均值。由于arr.mean(0)的长度为3，所以它可以在0轴向上进行广播：因为arr的后缘维度是3，所以它们是兼容的。根据该原则，要在1轴向上做减法（即各行减去行平均值），较小的那个数组的形状必须是(4,1)：

In [24]:
arr

array([[ 1.46367305e+00,  5.49854060e-01,  1.41297481e+00],
       [ 1.15477989e+00, -8.49189952e-01, -1.95989985e+00],
       [-1.58178574e+00, -7.07576930e-01, -1.89393709e+00],
       [ 9.76919327e-01,  4.48428159e-01,  9.93296455e-04]])

In [27]:
row_means = arr.mean(1)


row_means.shape

(4,)

In [28]:
row_means.reshape((4, 1))

array([[ 1.14216731],
       [-0.55143664],
       [-1.39443325],
       [ 0.47544693]])

In [30]:
demeaned = arr - row_means.reshape((4, 1))

demeaned.mean(1)

array([ 1.11022302e-16,  7.40148683e-17, -7.40148683e-17, -3.70074342e-17])

图A-5说明了该运算的过程。

二维数组在轴1上的广播

图A-6展示了另外一种情况，这次是在一个三维数组上沿0轴向加上一个二维数组。

三维数组在轴0上的广播

## 沿其它轴向广播

高维度数组的广播似乎更难以理解，而实际上它也是遵循广播原则的。如果不然，你就会得到下面这样一个错误：

In [31]:
arr - arr.mean(1)

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

人们经常需要通过算术运算过程将较低维度的数组在除0轴以外的其他轴向上广播。根据广播的原则，较小数组的“广播维”必须为1。在上面那个行距平化的例子中，这就意味着要将行平均值的形状变成(4,1)而不是(4,)：

In [32]:
arr - arr.mean(1).reshape((4, 1))

array([[ 0.32150574, -0.59231325,  0.2708075 ],
       [ 1.70621653, -0.29775332, -1.40846321],
       [-0.18735249,  0.68685632, -0.49950383],
       [ 0.5014724 , -0.02701877, -0.47445363]])

对于三维的情况，在三维中的任何一维上广播其实也就是将数据重塑为兼容的形状而已。图A-7说明了要在三维数组各维度上广播的形状需求。

能在该三维数组上广播的二维数组的形状

于是就有了一个非常普遍的问题（尤其是在通用算法中），即专门为了广播而添加一个长度为1的新轴。虽然reshape是一个办法，但插入轴需要构造一个表示新形状的元组。这是一个很郁闷的过程。因此，NumPy数组提供了一种通过索引机制插入轴的特殊语法。下面这段代码通过特殊的np.newaxis属性以及“全”切片来插入新轴：

In [34]:
arr = np.zeros((4, 4))


arr_3d = arr[:, np.newaxis, :]


arr_3d.shape

(4, 1, 4)

In [36]:
arr_1d = np.random.normal(size=3)

arr_1d[:, np.newaxis]

array([[ 0.70350853],
       [-0.61439544],
       [ 0.98136066]])

In [37]:
arr_1d[np.newaxis, :]

array([[ 0.70350853, -0.61439544,  0.98136066]])

因此，如果我们有一个三维数组，并希望对轴2进行距平化，那么只需要编写下面这样的代码就可以了：

In [39]:
arr = np.random.randn(3, 4, 5)

depth_means = arr.mean(2)

depth_means

array([[ 0.0122237 ,  0.49249001, -0.01649733,  1.52224706],
       [ 0.74515135,  0.1556852 ,  0.25522562, -0.13554703],
       [ 0.44620237,  0.846375  , -0.21527686,  0.42231453]])

In [40]:
depth_means.shape

(3, 4)

In [41]:
demeaned = arr - depth_means[:, :, np.newaxis]

demeaned.mean(2)

array([[-2.35922393e-17,  0.00000000e+00,  4.44089210e-17,
        -1.33226763e-16],
       [ 8.88178420e-17,  4.44089210e-17,  0.00000000e+00,
         2.22044605e-17],
       [ 0.00000000e+00,  0.00000000e+00, -2.22044605e-17,
        -8.88178420e-17]])

有些读者可能会想，在对指定轴进行距平化时，有没有一种既通用又不牺牲性能的方法呢？实际上是有的，但需要一些索引方面的技巧：

In [42]:
def demean_axis(arr, axis=0):
    means = arr.mean(axis)

    # This generalizes things like [:, :, np.newaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]

## 通过广播设置数组的值

算术运算所遵循的广播原则同样也适用于通过索引机制设置数组值的操作。对于最简单的情况，我们可以这样做：

In [43]:
arr = np.zeros((4, 3))


arr[:] = 5


arr

array([[5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.]])

但是，假设我们想要用一个一维数组来设置目标数组的各列，只要保证形状兼容就可以了：

In [44]:
col = np.array([1.28, -0.42, 0.44, 1.6])


arr[:] = col[:, np.newaxis]


arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [45]:
arr[:2] = [[-1.37], [0.509]]

arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])